In [1]:
import os
import sys
import glob 
import random
import warnings
from tqdm import tqdm

import numpy as np
import pandas as pd
from statsmodels import robust

import librosa
import librosa.display

import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import IPython.display as ipd 

import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
#initializing all the file path to a variable
AUDIO_PATH = r'C:\Users\moham\Desktop\esc50 and YBSS-200'
CSV_PATH =  r'C:\Users\moham\Desktop\noise_classification\csv files\esc50andYBSS200.csv'
CSV_PATH1 = r'C:\Users\moham\Desktop\noise_classification\csv files\first5keywords.csv'
CSV_PATH2 = r'C:\Users\moham\Desktop\noise_classification\csv files\first10keywords.csv'
CSV_PATH3 = r'C:\Users\moham\Desktop\noise_classification\csv files\first15keywords.csv'
CSV_PATH4 = r'C:\Users\moham\Desktop\noise_classification\csv files\YBSS_dataset.csv'
CSV_PATH5 = r'C:\Users\moham\Desktop\noise_classification\csv files\esc50new.csv'

## Part 1: Primary data analysis


In [3]:
df = pd.read_csv(CSV_PATH3)
print("shape of df: ", df.shape)
df.head()

#checking the content of the file

shape of df:  (600, 7)


,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-101336-A-30.wav,1,30,door_wood_knock,False,101336,A
3,1-103995-A-30.wav,1,30,door_wood_knock,False,103995,A
4,1-103999-A-30.wav,1,30,door_wood_knock,False,103999,A


In [4]:
df=df.drop(['fold','esc10','src_file','take'], axis=1)
print("shape of df: ", df.shape)
print(df)
#filtering the table and removing the unwanted data

shape of df:  (600, 3)
              filename  target         category
0     1-100032-A-0.wav       0              dog
1    1-100038-A-14.wav      14   chirping_birds
2    1-101336-A-30.wav      30  door_wood_knock
3    1-103995-A-30.wav      30  door_wood_knock
4    1-103999-A-30.wav      30  door_wood_knock
..                 ...     ...              ...
595  5-263490-A-25.wav      25        footsteps
596  5-263491-A-25.wav      25        footsteps
597  5-263501-A-25.wav      25        footsteps
598   5-51149-A-25.wav      25        footsteps
599     5-9032-A-0.wav       0              dog

[600 rows x 3 columns]


In [5]:
#check NAS
df.isna().sum()
#checking for any file empty 

filename    0
target      0
category    0
dtype: int64

In [6]:
NUM_CLASSES = len(df['category'].unique())
len(df['category'].unique())

#printing all the unique catagories 

15

In [7]:
df['target'].value_counts()

#checking if all classes have 40 audio files

0     40
14    40
30    40
47    40
31    40
32    40
16    40
25    40
43    40
29    40
7     40
24    40
21    40
5     40
33    40
Name: target, dtype: int64

In [8]:
len(df) == len(df['filename'].unique())

True

In [9]:
classes = df['category'].unique()
classes

array(['dog', 'chirping_birds', 'door_wood_knock', 'airplane',
       'mouse_click', 'keyboard_typing', 'wind', 'footsteps', 'car_horn',
       'drinking_sipping', 'insects', 'coughing', 'sneezing', 'cat',
       'door_wood_creaks'], dtype=object)

## Part 2: Analysis of audio tracks

In [10]:
## an instance of each class
unique_class_df = df.drop_duplicates(subset=['target'])
unique_class_df.head()
classes = df['category'].unique()
class_dict = {i:x for x,i in enumerate(classes)}
df['target'] = df['category'].map(class_dict)
df


,filename,target,category
0,1-100032-A-0.wav,0,dog
1,1-100038-A-14.wav,1,chirping_birds
2,1-101336-A-30.wav,2,door_wood_knock
3,1-103995-A-30.wav,2,door_wood_knock
4,1-103999-A-30.wav,2,door_wood_knock
...,...,...,...
595,5-263490-A-25.wav,7,footsteps
596,5-263491-A-25.wav,7,footsteps
597,5-263501-A-25.wav,7,footsteps
598,5-51149-A-25.wav,7,footsteps


In [14]:
X = []
y = []
for data in tqdm(df.iterrows(),  desc='Progress'):
    sig , sr = librosa.load(AUDIO_PATH + "\\" + data[1][0],sr = 44100)
    #taking 3 windows from 0-2 , 2-4 , 3-5 seconds
    sig_ = sig[0 : 88200]
    if robust.mad(sig_, axis=0) > 1e-4:
        mfcc_ = librosa.feature.mfcc(y=sig_ , sr=sr,hop_length = 512, n_mfcc=40)
        X.append(mfcc_)
        y.append(data[1][1])
    sig_ = sig[88200 : 176400]
    if robust.mad(sig_, axis=0) > 1e-4:
        mfcc_ = librosa.feature.mfcc(y=sig_ , sr=sr,hop_length = 512, n_mfcc=40)
        X.append(mfcc_)
        y.append(data[1][1])
    sig_ = sig[132300 : 220500]
    if robust.mad(sig_, axis=0) > 1e-4:
        mfcc_ = librosa.feature.mfcc(y=sig_ , sr=sr,hop_length = 512, n_mfcc=40)
        X.append(mfcc_)
        y.append(data[1][1])

Progress: 0it [00:00, ?it/s]C:\Users\moham\AppData\Local\Temp\ipykernel_16408\2922706038.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  sig , sr = librosa.load(AUDIO_PATH + "\\" + data[1][0],sr = 44100)
Progress: 0it [00:00, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\moham\\Desktop\\esc50 and YBSS-200\\1-100032-A-0.wav'

In [ ]:
import matplotlib.pyplot as plt
import librosa.display
plt.figure(figsize=(10,3))
plt.title("Single Bark Wave Plot")
librosa.display.waveshow(sig_,sr)

In [ ]:
X = np.array(X) 
y = np.array(y)

In [ ]:
print("X Shape is: ", X.shape)
print("y Shape is: ", y.shape)

In [ ]:
y = tf.keras.utils.to_categorical(y , num_classes=15)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X , y ,test_size=0.2, random_state=42)

In [ ]:
INPUTSHAPE = (40,173,1)

In [ ]:
y.shape

In [ ]:
model =  models.Sequential([
    
  layers.Conv2D(32 , (3,3),activation = 'relu',padding='valid', input_shape = INPUTSHAPE),  
  layers.MaxPooling2D(2, padding='same'),
  layers.Conv2D(128, (3,3), activation='relu',padding='valid'),
  layers.MaxPooling2D(2, padding='same'),
  layers.Dropout(0.3),
  layers.Conv2D(128, (3,3), activation='relu',padding='valid'),
  layers.MaxPooling2D(2, padding='same'),
  layers.Dropout(0.3),
  layers.GlobalAveragePooling2D(),
  
  layers.Dense(512 , activation = 'relu'),
  layers.Dense(15, activation = 'softmax')
])

model.compile(loss = 'categorical_crossentropy', optimizer = 'Nadam', metrics = 'acc')
model.summary()

In [ ]:
batch_size = 15
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=8, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False)

history = model.fit(X_train,y_train,
            validation_data=(X_test,y_test),
            epochs=25,
            callbacks = [callback],batch_size=batch_size)

Metrics





In [ ]:
pred = model.predict(X_test)

In [ ]:
np.argmax(y_test,1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(np.argmax(y_test,1), np.argmax(pred,1), target_names=classes))
#number of records in the test set=support

### Сonfusion matrix

Using this matrix, you can understand in which classes


model is wrong. (i.e. which class gets confused with the other)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(np.argmax(y_test,1), np.argmax(pred,1))

In [ ]:
import seaborn as snNew
fig, ax = plt.subplots(figsize=(30,30))
snNew.heatmap(cm, annot=True, xticklabels= classes, yticklabels= classes)

In [ ]:
x1 = []
y1 = []
for data in tqdm(df.iterrows(),  desc='Progress'):
        sig , sr = librosa.load(AUDIO_PATH + "\\" + data[1][0],sr = 44100)
        sig_ = sig[0 : 110250]
        mfcc_ = librosa.feature.mfcc(sig_ , sr=sr,hop_length = 640 , n_mfcc=40,dct_type=2)
        x1.append(mfcc_)
        y1.append(data[1][1])
    

In [ ]:
x1 = np.array(x1) 
y1 = np.array(y1)

In [ ]:
print("X Shape is: ", x1.shape)
print("y Shape is: ", y1.shape)

In [ ]:
y1 = tf.keras.utils.to_categorical(y1 , num_classes=50)
x1 = x1.reshape(x1.shape[0], x1.shape[1], x1.shape[2], 1)

In [ ]:
pred = model.predict(x1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(np.argmax(y1,1), np.argmax(pred,1), target_names=classes))

In [ ]:
test_path = r"C:\Users\harsh\Desktop\concatenated audio(15 noises)"

In [ ]:
concat_test = []

In [ ]:
for i in tqdm(file_names):
    sig , sr = librosa.load(test_path + "\\" + i,sr = 44100)
    sig_ = sig[0 : 441000]
    mfcc_ = librosa.feature.mfcc(sig_ , sr=sr,hop_length = 2550 , n_mfcc=40,dct_type=2)
    concat_test.append(mfcc_)
    #y1.append(data[1][1])
    

In [ ]:
len(sig)

In [ ]:
concat_test = np.array(concat_test)

In [ ]:
concat_test = concat_test.reshape(concat_test.shape[0], concat_test.shape[1], concat_test.shape[2], 1)

In [ ]:
concat_test.shape

In [ ]:
concat_test = np.array(concat_test)

In [ ]:
prediction = model.predict(concat_test)

In [ ]:
prediction_result=[]
for i in classes[np.argmax(prediction,1)]:
    prediction_result.append(i)

In [ ]:
for i in classes[np.argmax(prediction,1)]:
    print(i)

In [ ]:
prediction_result

In [ ]:
count = 0
for i in range(0,len(prediction_result)):
    w = file_names[i].split(".")[0]
    t = w.split("_and_")
    if prediction_result[i] in t:
        print(1)
        count = count+1
    else:
        print(0)